**D3APL: Aplicações em Ciência de Dados** <br/>
IFSP Campinas

Prof. Dr. Samuel Martins (Samuka) <br/><br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# Regression Multilayer Perceptron (MLP) with Keras - V2
In this notebook, we will see how to wrap a Keras Model to be used in Sklearn environment. <br/>
We'll see:
- `cross_val_score`

Instead of fixing a Validation Set as did before, we will use `cross_val_score`.

## 1. Set up

#### 1.1 TensorFlow + Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras

### 1.2 Fixing the seed for reproducibility (optional)
That's a try for reprodubility in Keras. See more on: <br/>
https://machinelearningmastery.com/reproducible-results-neural-networks-keras/ <br/>
https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In [ ]:
from numpy.random import seed
from tensorflow.random import set_seed

seed(42)
set_seed(42)

#### 1.3 Other imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#### 1.4 Loading California Housing Dataset via Scikit-learn
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

In [ ]:
X_train = np.load('./datasets/housing_train_data.npy')
y_train = np.load('./datasets/housing_train_labels.npy')
X_test = np.load('./datasets/housing_test_data.npy')
y_test = np.load('./datasets/housing_test_labels.npy')

In [ ]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}\n')

print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}\n')

#### 1.5 Normalization
For simplicity, we are going to use the `StandardScaler()` without taking into account possible problems with outliers, etc.

When normalizing the dataset before, our _cross-validation_ results will suffer a little from **data-snooping bias**.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## 2. Wrapping Keras Model to Sklearn

### 2.1 Defining the Network Architecture
Proposed architecture for Multiclass Classification:
- Input Layer: 8 neurons
- Hidden Layer: 30 neurons, ReLu
- Output Layer: 1 neuron (no activation)**


In short: [8], [30 ReLU], [1]

#### **Wrapping**

##### **A function that builds a Keras Model**
Create a function that returns a **Keras model**, taking as _parameters_ things you want to verify using _cross-valdiation_ and _model selection_.

https://stackoverflow.com/a/44134591/7069696 <br/>
https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

##### **Wrapping**

**Deprecated**: <br/>
https://faroit.com/keras-docs/1.2.2/scikit-learn-api/

**Current alternative - SciKeras** <br/>
https://github.com/adriangb/scikeras <br/>
https://www.adriangb.com/scikeras/stable/migration.html

In [ ]:
# !pip install scikeras[tensorflow]

### 2.2 Cross-validation
https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

Observations:
- My experiments are using a GPU drive which has limited memory. Therefore, I am using only a single job for cross-validation.
- The score functions is `neg_mean_squared_error` just to have the notion of _"the lower, the better"_. This is crucial for **fine-tunning**.

In case of GPU drivers, we can monitor its use by [_gpustat_](https://github.com/wookayin/gpustat).

On terminal, use: `gpustat -cpi`


In [ ]:
# one score for each fold
